<a href="https://colab.research.google.com/github/yamkas/Advance-SQL-Syntax-/blob/master/Advance_SQL_Syntax_(for_students).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Persiapan

In [ ]:
# import library yang dibutuhkan
import sqlite3
import pandas as pd

In [ ]:
#upload file
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# cek versi dari sqlite kita
sqlite3.sqlite_version

'3.37.2'

In [ ]:
# membuat koneksi ke database
db = sqlite3.connect('northwind.db')

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = db.cursor()

In [ ]:
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


## 2. SUB QUERY

In [ ]:
#Mengambil data pelanggan yang telah melakukan pembelian tahun 2014 dengan JOIN

query = """
select
a.customerid,
a.companyname,
a.contactname
from customers a
left join orders b
where date(b.orderdate,'start of year') = '2014-01-01'
"""

pd.read_sql_query (query , db)

,CustomerID,CompanyName,ContactName
0,ALFKI,Alfreds Futterkiste,Maria Anders
1,ALFKI,Alfreds Futterkiste,Maria Anders
2,ALFKI,Alfreds Futterkiste,Maria Anders
3,ALFKI,Alfreds Futterkiste,Maria Anders
4,ALFKI,Alfreds Futterkiste,Maria Anders
...,...,...,...
13827,WOLZA,Wolski Zajazd,Zbyszek Piestrzeniewicz
13828,WOLZA,Wolski Zajazd,Zbyszek Piestrzeniewicz
13829,WOLZA,Wolski Zajazd,Zbyszek Piestrzeniewicz
13830,WOLZA,Wolski Zajazd,Zbyszek Piestrzeniewicz


In [ ]:
#Mengambil data pelanggan yang telah melakukan pembelian tahun 2014 dengan Subquery (KLAUSA WHERE)

query = """
select customerid,
companyname,
contactname
from customers
where customerid in
(select distinct customerid from orders
where date(orderdate,'start of year') = '2014-01-01')
"""

pd.read_sql_query (query , db)

,CustomerID,CompanyName,ContactName
0,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo
1,ANTON,Antonio Moreno Taquería,Antonio Moreno
2,AROUT,Around the Horn,Thomas Hardy
3,BERGS,Berglunds snabbköp,Christina Berglund
4,BLONP,Blondesddsl père et fils,Frédérique Citeaux
...,...,...,...
62,WANDK,Die Wandernde Kuh,Rita Müller
63,WARTH,Wartian Herkku,Pirkko Koskitalo
64,WELLI,Wellington Importadora,Paula Parente
65,WHITC,White Clover Markets,Karl Jablonski


In [ ]:
#Menghitung total transaksi di setiap kota tahun 2014 (KLAUSA FROM)

query = """
select
shipcity,
count(distinct orderid) as total_order
from
(
  select shipcity, orderid
  from orders
  where date(orderdate,'start of year') = '2014-01-01')
group by 1
order by 2 desc,1
limit 10
"""

pd.read_sql_query (query , db)

,shipcity,total_order
0,México D.F.,9
1,Rio de Janeiro,7
2,Albuquerque,6
3,Cunewalde,6
4,Graz,6
5,Barquisimeto,5
6,Cork,5
7,Lander,5
8,London,5
9,Sao Paulo,5


In [ ]:
#Menghitung total transaksi di setiap kota tahun 2014 (KLAUSA SELECT)

query = """
select
city,
(
  select count(distinct orderid)
  from orders
  where date(orderdate,'start of year') = '2014-01-01'
  and orders.shipcity = customers.city) as total_pembelian
from customers
group by 1
order by 2 desc,1
limit 10
"""

pd.read_sql_query (query , db)

,City,total_pembelian
0,México D.F.,9
1,Rio de Janeiro,7
2,Albuquerque,6
3,Cunewalde,6
4,Graz,6
5,Barquisimeto,5
6,Cork,5
7,Lander,5
8,London,5
9,Sao Paulo,5


In [ ]:
#cek perhoitungan total transaksi di setiap kota tahun 2014

query = """
select shipcity, count(orderid) from orders
where date(orderdate,'start of year') = '2014-01-01'
group by 1
order by 2 desc,1
limit 10
"""

pd.read_sql_query (query , db)

,ShipCity,count(orderid)
0,México D.F.,9
1,Rio de Janeiro,7
2,Albuquerque,6
3,Cunewalde,6
4,Graz,6
5,Barquisimeto,5
6,Cork,5
7,Lander,5
8,London,5
9,Sao Paulo,5


## CTE

In [ ]:
#Menghitung total transaksi di setiap kota tahun 2014 (KLAUSA FROM)

query = """
with cte_order_per_kota as (
  select shipcity, count(distinct orderid) as total_order
  from orders
  where date(orderdate,'start of year') = '2014-01-01'
  group by 1)
select * from cte_order_per_kota
order by 2 desc,1
limit 10
"""

pd.read_sql_query (query , db)

,shipcity,total_order
0,México D.F.,9
1,Rio de Janeiro,7
2,Albuquerque,6
3,Cunewalde,6
4,Graz,6
5,Barquisimeto,5
6,Cork,5
7,Lander,5
8,London,5
9,Sao Paulo,5


In [ ]:
#Memunculkan customer yang yang bertransaksi tahun 2014 (KLAUSA WHERE)

query = """
with cte_order_per_kota as (
  select customerid from  orders
  where date(orderdate,'start of year') = '2014-01-01')
select * from customers where customerid in (select customerid from cte_order_per_kota)
"""

pd.read_sql_query (query , db)

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
1,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
2,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
3,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
4,BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,None,67000,France,88.60.15.31,88.60.15.32
...,...,...,...,...,...,...,...,...,...,...,...
62,WANDK,Die Wandernde Kuh,Rita Müller,Sales Representative,Adenauerallee 900,Stuttgart,None,70563,Germany,0711-020361,0711-035428
63,WARTH,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
64,WELLI,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
65,WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115


In [ ]:
#Memunculkan customer yang yang bertransaksi tahun 2014 (KLAUSA WHERE)

query = """
select * from orders
"""

pd.read_sql_query (query , db)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,2014-07-08 14:00:00.000,2014-08-05 00:00:00.000,2014-07-15 00:00:00.000,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,2014-07-09 01:00:00.000,2014-08-06 00:00:00.000,2014-07-11 00:00:00.000,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,2016-05-05 12:00:00.000,2016-06-02 00:00:00.000,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico
826,11074,SIMOB,7,2016-05-06 01:00:00.000,2016-06-03 00:00:00.000,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,2016-05-06 18:00:00.000,2016-06-03 00:00:00.000,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland
828,11076,BONAP,4,2016-05-06 00:00:00.000,2016-06-03 00:00:00.000,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


In [ ]:
# CTE Bertingkat

query = """

with cte_products as (
  select
  productid,
  productname
  from products),

cte_orderdetails as (
select
productid,
sum(unitprice*quantity) as total_penjualan
from
orderdetails
group by 1
)

SELECT
a.productid,
a.productname,
b.total_penjualan
from cte_products a
left join cte_orderdetails b
on a.productid = b.productid
order by 3
"""
pd.read_sql(query, db)

,productid,productname,total_penjualan
0,48,Chocolade,1542.75
1,33,Geitost,1713.50
2,15,Genen Shouyu,1813.50
3,67,Laughing Lumberjack Lager,2562.00
4,74,Longlife Tofu,2798.00
...,...,...,...
72,62,Tarte au sucre,49827.90
73,60,Camembert Pierrot,50286.00
74,59,Raclette Courdavault,76296.00
75,29,Thüringer Rostbratwurst,87736.40


# Mini Case

Selamat ! Kamu baru saja diterima bekerja sebagai seorang Jr. Data Analyst di perusahaan Nortwind Traders!. Hari ini adalah hari pertama kamu bekerja, dan manager kamu ingin kamu secara perlahan mengetahui seluk beluk dari perusahaan tempat kamu bekerja. Kamu diminta untuk mengerjakan beberapa tugas berikut menggunakan bantuan SQL yang sudah kamu pelajari sebelumnya.
Tugas kamu adalah membuat query untuk menjawab beberapa tugas yang akan diberikan di bawah ini, kemudian kamu dapat memberikan komentar sesuai dengan hasil query kamu.
Selamat bekerja!!

Agar query mudah dibaca saya mencoba untuk menggunakan CTE bertingkat, data yang dibutuhkan adalah:
1. Table products: productid, productname
2. Table Orderdetails: productid, orderid, total penjualan
3. Table orders: orderid, orderdate, shippeddate
4. Gabungkan table a,b,c menggunakan LEFT JOIN

In [ ]:
#NOTE NOMOR 1 menggunakan fungsi USING dengan nama kolom ID yang sama
query = """
 with cte_produk as (
  select
  productid,
  productname
  from products),

cte_orderdetails as(
  select
  productid,
  orderid,
  sum(unitprice*quantity) as total_penjualan
  from
  orderdetails
  group by 1,2
),

cte_orders as(
  select
  orderid,
  orderdate,
  shippeddate
  from
  orders
)

SELECT
cte_produk.productid,
cte_produk.productname,
cte_orderdetails.total_penjualan,
cte_orders.orderid,
cte_orders.orderdate,
cte_orders.shippeddate
from cte_produk
left join cte_orderdetails using(productid)
left join cte_orders using(orderid)
order by 3
"""
pd.read_sql(query, db)

,productid,productname,total_penjualan,orderid,orderdate,shippeddate
0,13,Konbu,4.8,10462,2015-03-03 08:00:00.000,2015-03-18 00:00:00.000
1,19,Teatime Chocolate Biscuits,7.3,10281,2014-08-14 14:00:00.000,2014-08-21 00:00:00.000
2,13,Konbu,9.6,10420,2015-01-21 08:00:00.000,2015-01-27 00:00:00.000
3,33,Geitost,10.0,10850,2016-01-23 22:00:00.000,2016-01-30 00:00:00.000
4,31,Gorgonzola Telino,12.5,10782,2015-12-17 13:00:00.000,2015-12-22 00:00:00.000
...,...,...,...,...,...,...
2150,38,Côte de Blaye,10540.0,10353,2014-11-13 12:00:00.000,2014-11-25 00:00:00.000
2151,38,Côte de Blaye,10540.0,10417,2015-01-16 07:00:00.000,2015-01-28 00:00:00.000
2152,38,Côte de Blaye,10540.0,10889,2016-02-16 04:00:00.000,2016-02-23 00:00:00.000
2153,38,Côte de Blaye,15810.0,10865,2016-02-02 04:00:00.000,2016-02-12 00:00:00.000


In [ ]:
#NOTE NOMOR 1 menggunakan fungsi On
query = """
 with cte_produk as (
  select
  productid,
  productname
  from products),

cte_orderdetails as(
  select
  productid,
  orderid,
  sum(unitprice*quantity) as total_penjualan
  from
  orderdetails
  group by 1,2
),

cte_orders as(
  select
  orderid,
  orderdate,
  shippeddate
  from
  orders
)

SELECT
cte_produk.productid,
cte_produk.productname,
cte_orderdetails.total_penjualan,
cte_orders.orderid,
cte_orders.orderdate,
cte_orders.shippeddate
from cte_produk
left join cte_orderdetails on cte_produk.productid = cte_orderdetails.productid
left join cte_orders on cte_orders.orderid = cte_orderdetails.orderid
order by 3
"""
pd.read_sql(query, db)

,productid,productname,total_penjualan,orderid,orderdate,shippeddate
0,13,Konbu,4.8,10462,2015-03-03 08:00:00.000,2015-03-18 00:00:00.000
1,19,Teatime Chocolate Biscuits,7.3,10281,2014-08-14 14:00:00.000,2014-08-21 00:00:00.000
2,13,Konbu,9.6,10420,2015-01-21 08:00:00.000,2015-01-27 00:00:00.000
3,33,Geitost,10.0,10850,2016-01-23 22:00:00.000,2016-01-30 00:00:00.000
4,31,Gorgonzola Telino,12.5,10782,2015-12-17 13:00:00.000,2015-12-22 00:00:00.000
...,...,...,...,...,...,...
2150,38,Côte de Blaye,10540.0,10353,2014-11-13 12:00:00.000,2014-11-25 00:00:00.000
2151,38,Côte de Blaye,10540.0,10417,2015-01-16 07:00:00.000,2015-01-28 00:00:00.000
2152,38,Côte de Blaye,10540.0,10889,2016-02-16 04:00:00.000,2016-02-23 00:00:00.000
2153,38,Côte de Blaye,15810.0,10865,2016-02-02 04:00:00.000,2016-02-12 00:00:00.000


Masih dengan CTE yang sama:
1. coba filter hanya order di tahun 2015,
2. urutkan berdasarkan total penjualan terbesar,
3. ambil top 10 saja

In [ ]:
# NOMOR 2
query = """
 with cte_produk as (
  select
  productid,
  productname
  from products),

cte_orderdetails as(
  select
  productid,
  orderid,
  sum(unitprice*quantity) as total_penjualan
  from
  orderdetails
  group by 1,2
),

cte_orders as(
  select
  orderid,
  orderdate,
  shippeddate
  from
  orders
)

SELECT
cte_produk.productid,
cte_produk.productname,
cte_orderdetails.total_penjualan,
cte_orders.orderid,
cte_orders.orderdate,
cte_orders.shippeddate
from cte_produk
left join cte_orderdetails on cte_produk.productid = cte_orderdetails.productid
left join cte_orders on cte_orders.orderid = cte_orderdetails.orderid
WHERE date(cte_orders.orderdate,'start of year') = '2015-01-01'
group by 1,2
order by 3 DESC
LIMIT 10
"""
pd.read_sql(query, db)

,productid,productname,total_penjualan,orderid,orderdate,shippeddate
0,38,Côte de Blaye,10540.0,10417,2015-01-16 07:00:00.000,2015-01-28 00:00:00.000
1,20,Sir Rodney's Marmalade,3159.0,10514,2015-04-22 08:00:00.000,2015-05-16 00:00:00.000
2,63,Vegie-spread,2281.5,10402,2015-01-02 18:00:00.000,2015-01-10 00:00:00.000
3,56,Gnocchi di nonna Alice,2128.0,10401,2015-01-01 15:00:00.000,2015-01-10 00:00:00.000
4,29,Thüringer Rostbratwurst,2079.0,10400,2015-01-01 21:00:00.000,2015-01-16 00:00:00.000
5,27,Schoggi Schokolade,1755.0,10441,2015-02-10 16:00:00.000,2015-03-14 00:00:00.000
6,6,Grandma's Boysenberry Spread,1750.0,10618,2015-08-01 04:00:00.000,2015-08-08 00:00:00.000
7,60,Camembert Pierrot,1632.0,10419,2015-01-20 18:00:00.000,2015-01-30 00:00:00.000
8,32,Mascarpone Fabioli,1600.0,10513,2015-04-22 05:00:00.000,2015-04-28 00:00:00.000
9,9,Mishi Kobe Niku,1552.0,10420,2015-01-21 08:00:00.000,2015-01-27 00:00:00.000


Buat 3 CTE dan gabungkan menggunakan union
1. cte1: table orderdetails --> productid, total penjualan hanya product ('38','20','63')
2. cte1: table orderdetails --> productid, total penjualan 110% hanya product ('38','20','63')
3. cte1: table orderdetails --> productid, total penjualan 120% hanya product ('38','20','63')


In [ ]:
#NOMOR 3
query = """

with cte_orderdetails as (
select
productid,
sum(unitprice*quantity) as total_penjualan
from
orderdetails
where productid in ('38','20','63')
group by 1
),

cte_orderdetails_110 as (
select
productid,
sum(unitprice*quantity)*1.1 as total_penjualan
from
orderdetails
where productid in ('38','20','63')
group by 1),

cte_orderdetails_120 as (
select
productid,
sum(unitprice*quantity)*1.2 as total_penjualan
from
orderdetails
where productid in ('38','20','63')
group by 1)

SELECT * from cte_orderdetails
union all
SELECT * from cte_orderdetails_110
union all
SELECT * from cte_orderdetails_120
order by 1
"""
pd.read_sql(query, db)

,productid,total_penjualan
0,20,23635.80
1,20,25999.38
2,20,28362.96
3,38,149984.20
4,38,164982.62
5,38,179981.04
6,63,17696.30
7,63,19465.93
8,63,21235.56
